# FBGEMM FP8（模型量化至权重为8位，激活为8位）

通过使用 FBGEMM FP8 量化方法，你可以将模型量化为 FP8（权重为 8 位，激活为 8 位）：

- 权重将按通道量化为 8 位（FP8）
- 激活将按 token 量化为 8 位（FP8）

该方法依赖于 [FBGEMM](https://github.com/pytorch/FBGEMM) 库，该库提供了高效的小批量低精度通用矩阵乘法，并支持行级量化和异常值感知量化等减少精度损失的技术。

你需要一个计算能力 >= 9 的 GPU（例如 H100）。

在开始之前，请确保安装了以下库的最新版本：


In [ ]:
pip install --upgrade accelerate fbgemm-gpu torch


如果安装 `fbgemm-gpu` 和 `torch` 库时遇到问题，可能需要安装夜间版。你可以按照 [这里的说明](https://pytorch.org/FBGEMM/fbgemm_gpu-development/InstallationInstructions.html#fbgemm-gpu-install-libraries:~:text=found%20here.-,Install%20the%20FBGEMM_GPU%20Package,-Install%20through%20PyTorch) 进行操作。

默认情况下，无论权重实际存储的数据类型（如 `torch.float16`），权重都会以全精度（`torch.float32`）加载。设置 `torch_dtype="auto"` 可以根据模型的 `config.json` 文件中定义的数据类型自动加载最节省内存的数据类型。


In [ ]:
from transformers import FbgemmFp8Config, AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B"
quantization_config = FbgemmFp8Config()
quantized_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
input_text = "我们今晚吃什么？"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=10)
print(tokenizer.decode(output[0], skip_special_tokens=True))


量化后的模型可以通过 `save_pretrained` 保存，并通过 `from_pretrained` 重新加载使用。


In [ ]:
quant_path = "/path/to/save/quantized/model"
model.save_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="auto")